In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np
import dill as dpickle

In [ ]:
with open('title_pp.dpkl', 'rb') as f:
    title_pp = dpickle.load(f)

with open('body_pp.dpkl', 'rb') as f:
    body_pp = dpickle.load(f)
    
#load the training data and labels
train_body_vecs = np.load('train_body_vecs.npy')
train_title_vecs = np.load('train_title_vecs.npy')
train_labels = np.load('train_labels.npy')

#load the test data and labels
test_body_vecs = np.load('test_body_vecs.npy')
test_title_vecs = np.load('test_title_vecs.npy')
test_labels = np.load('test_labels.npy')


issue_body_doc_length = train_body_vecs.shape[1]
issue_title_doc_length = train_title_vecs.shape[1]

body_vocab_size = body_pp.n_tokens
title_vocab_size = title_pp.n_tokens

num_classes = len(set(train_labels[:, 0]))

In [ ]:
body_input = Input(shape=(issue_body_doc_length,), name='Body-Input')
title_input = Input(shape=(issue_title_doc_length,), name='Title-Input')

body = Embedding(body_vocab_size, 50, name='Body-Embedding')(body_input)
title = Embedding(title_vocab_size, 50, name='Title-Embedding')(title_input)

body = BatchNormalization()(body)
body = GRU(100, name='Body-Encoder')(body)

title = BatchNormalization()(title)
title = GRU(75, name='Title-Encoder')(title)

x = Concatenate(name='Concat')([body, title])
x = BatchNormalization()(x)
out = Dense(num_classes, activation='softmax')(x)

model = Model([body_input, title_input], out)

model.compile(optimizer=Adam(lr=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

script_name_base = 'Issue_Label_v1'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}_best_model.hdf5'.format(script_name_base),
                                   save_best_only=True)

In [ ]:
batch_size = 900
epochs = 4
history = model.fit(x=[train_body_vecs, train_title_vecs], 
                    y=train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=[(test_body_vecs, test_title_vecs), test_labels], 
                    callbacks=[csv_logger, model_checkpoint])